<a href="https://colab.research.google.com/github/guanwee-loo/Notebooks/blob/master/SAML2_0_%E6%BC%8F%E6%B4%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # SAML 2.0 绕过认证漏洞
 
 卢传惠
 

# 什么是 SAML 2.0 ?

Security Assertion Markup Language 2.0 (SAML 2.0)是SAML标准的一个版本，用于在安全域之间交换身份验证和授权数据。 SAML 2.0是一种XML (Extensible Markup Language) 的协议，它使用包含断言的安全令牌 (security tokens)，在名为**身份提供商** (Identity Provider)和名为**服务提供商**(Service Provider)传递有关用户的身份和权限信息。SAML 2.0基于支持互联网应用程序的跨域单点登录（Single Sign-On），有助于减少向用户分发多个身份验证令牌的管理开销。


**当用户使用SAML对网站进行身份验证时，总会涉及三方:**

1.   浏览器中的用户
2.   运行用户尝试访问的网站的服务提供商（SP） (例如,  Salesforce)
3.   存储和管理用户帐户和凭据的身份提供者（IdP）（例如，Okta，OneLogin）

# 正常登录流程 
![alt text](https://raw.githubusercontent.com/guanwee-loo/Notebooks/master/WebSSOzh_cn.png)         





要掌握的重要概念是SAML响应对服务提供者（SP）的意义，以及如何处理它。

响应处理有很多细微之处，但简化版通常看起来像：

* 用户向身份提供者（IdP）进行身份验证，生成电子签名的SAML响应。然后，用户的浏览器将此响应转发给SP。

* SP验证SAML响应签名。

* 如果签名有效，SAML响应中的字符串标识符（例如NameID）将用来进行用户身的份验证。

# 什么是SAML 2.0 绕过认证漏洞？
它是一个新的漏洞类，它影响基于SAML的单点登录（SSO）系统。此漏洞可允许具有经过身份验证访问权限的攻击者欺骗SAML系统，以便在不知道受害者用户密码的情况下作为其他用户进行身份验证。

有3个成分可以实现此漏洞:

1. SAML响应包含标识身份验证用户的字符串。

2. XML规范化将删除注释（取决于配置）作为电子签名验证的一部分，因此向SAML响应添加注释不会使电子签名无效。

3. 当存在注释时，XML文本提取可能仅返回XML元素中文本的子字符串。


# 简化的SAML响应示例



```
<SAMLResponse>
    <Issuer>https://idp.com/</Issuer>
    <Assertion ID="_id1234">
        <Subject>
            <NameID>user@user.com</NameID>
        </Subject>
    </Assertion>
    <Signature>
        <SignedInfo>
            <CanonicalizationMethod Algorithm="xml-c14n11"/>
            <Reference URI="#_id1234"/>
        </SignedInfo>
        <SignatureValue> <!--电子签名 -->
            some base64 data that represents the signature of the assertion
        </SignatureValue>
    </Signature>
</SAMLRes
```

上面的XML blob中的两个基本元素是** Assertion **和** Signature **元素。

Assertion元素包含* NameID *元素。这是身份提供商（IdP）用来识别和验证即将登录的用户的字符串。为该断言元素生成电子签名，并将其存储为Signature元素的一部分，并由服务提供商（SP）使用确保数据完整性并防止修改NameID

# XML规范化 (C14N）

XML规范化允许两个逻辑上等效的XML文档具有相同的字节表示。例如：

```
<NameID>user@user.com.evil.com</NameID>
```

和


```
<NameID>user@user.com<!-- this is a comment -->.evil.com</NameID>
```
这两个文档具有不同的字节表示（第二个包含注释），但传达相同的信息（即它们在逻辑上是等价的）。

在电子签名之前，规范化应用于XML元素。这可以防止XML文档中的无意义差异导致不同的数字签名。
在上面的SAML响应中，规范化方法指定在签署文档之前应用哪种规范化方法。实践中最常见的算法似乎是http://www.w3.org/2001/10/xml-exc-c14n#。

** Python的lxml XML库（使用于开源SAML Python工具包“python3-saml”）可以证明上述行为（删除注释）**

In [0]:
from io import BytesIO
from io import StringIO

import lxml.etree as ET
NameID = StringIO("<NameID>user@user.com<!-- this is a comment -->.evil.com</NameID>")
tree = ET.parse(NameID)
buffer = BytesIO()
tree.write_c14n(buffer,with_comments=False) # DO NOT INCLUDE COMMENTS IN THE RESULT
print(buffer.getvalue().decode("utf-8"))

# 剥离评论后执行NameID电子签名将产生相同的签名 - 意想不到行为!!!!

<NameID>user@user.com.evil.com</NameID>


“解决方法”是使用具有标识符http://www.w3.org/2001/10/xml-exc-c14n#WithComments 的另一个exc-c14n变体.exc-c14n的这种变体不会省略注释，所以上面的两个XML文档虽然在逻辑上是等价的，但它们没有相同的规范表示。

In [0]:
from io import BytesIO
from io import StringIO

import lxml.etree as ET
NameID = StringIO("<NameID>user@user.com<!-- this is a comment -->.evil.com</NameID>")
tree = ET.parse(NameID)
buffer = BytesIO()
tree.write_c14n(buffer,with_comments=True) # INCLUDE COMMENTS IN THE RESULT
print(buffer.getvalue().decode("utf-8"))

# 在不剥离注释的情况下执行NameID的电子签名将产生不同的签名 - 预期行为

<NameID>user@user.com<!-- this is a comment -->.evil.com</NameID>


# XML文本提取

此漏洞的另一个原因是XML解析的微妙和意外行为。

考虑以下XML元素NameID：


```
<NameID> user@user.com<!-- this is a comment -->.evil.com </NameID>
```



In [0]:
from defusedxml.lxml import fromstring
resp = "<NameID>user@user.com<!-- this is a comment -->.evil.com</NameID>"
data = fromstring(resp)
print("解析的 NameID = " + data.text) 
# Expecting ‘user@user.com.evil.com’ but..
# it becomes 'user@user.com' after parsing. 

解析的 NameID = user@user.com


# 鉴于上述3个条件，攻击者可以执行以下操作以另一个用户身份登录：

1. 使用注册帐户或受感染帐户登录（NameID = user@user.com.evil.com）

2. IdP将电子签署包含NameID = user@user.com.evil.com的断言

3. 拦截SAML响应并将NameID更改为修改为:
     
   `  user@user.com<!--this is a comment -->.evil.com
   `
   
    XML C14N会在电子签名之前删除注释又把NameID变回user@user.com.evil.com,造成与他的原先登录的电子签名相同,所以无法防止签名无效。
     
4. 由于意外的XML文本提取行为，SP将错误地将攻击者识别为“user@user.com”，同时验证签名是否正确


# 攻击登录流程

![alt text](https://raw.githubusercontent.com/guanwee-loo/Notebooks/master/WebSSO_Attack_zn_cn.PNG)

# 缓解措施

* 禁用敏感网络上的用户帐户的公共注册，并手动审查每个用户，以避免攻击者首先在内部网络上注册帐户。

* 如果无法做到这一点，网络管理员可以配置接受的电子邮件地址域名的白名单，以限制谁可以在网络上注册。

* 实行双因素身份验证（2FA)。

* 其他可能的补救措施是更新库以使用C14N方法保留之前的注释或在电子签名验证之后使用规范化的XML文档进行任何处理。这可以防止此漏洞以及XML规范化问题可能引入的其他漏洞。

# 参考资料

1.   https://en.wikipedia.org/wiki/SAML_2.0
2.   https://developer.okta.com/blog/2018/02/27/a-breakdown-of-the-new-saml-authentication-bypass-vulnerability
3.   https://duo.com/blog/duo-finds-saml-vulnerabilities-affecting-multiple-implementations
4.   https://www.decalage.info/fr/python/lxml-c14n




